## Installing Required Libaries

In [1]:
pip install eli5

  Using cached eli5-0.13.0.tar.gz (216 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107728 sha256=83c173c911745c189a030b52b30abaeed466864db09392173023d6e65e7e5c5c
  Stored in directory: c:\users\marx\appdata\local\pip\cache\wheels\b8\58\ef\2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5
Note: you may need to restart the kernel to use updated packages.


## Importing the Required Libaries

In [151]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

## Loading the data

In [152]:
data = pd.read_csv('Churn_Modelling.csv')

## Choosing the Features

In [153]:
X = data.iloc[:,3:-1]

## Encoding the Categorical Features

In [154]:
## Our columns e.g Geography contains words and not numbers, These words can assume a limited number of values, called Categorical features
## Our ml model is mathematical so, it requires numbers for computation thats why we encode

In [155]:
encoder = OrdinalEncoder()
value = encoder.fit_transform(X['Geography'].values.reshape(-1, 1))
X['Geography'] = value
encoder = OrdinalEncoder()
value = encoder.fit_transform(X['Gender'].values.reshape(-1, 1))
X['Gender'] = value

In [156]:
## We've encoded the Geography and Gender columns with OrdinalEncoder()

## Getting the Target Column

In [157]:
## Predicting whether a customer leaves the bank is a supervised learning problem and so we have to train the model so as to be able to predict the right target variable which is a column of 0s and 1s .

In [158]:
y = data.iloc[:, len(data.columns)-1]

## Splitting the Data into Train and Test sets

In [159]:
## The function train_test_split will be used to divide our data into training and testing sets

In [160]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [161]:
## Checking the lengths of train and test sets in which the general practice requires 70% to be training and 30% to be testing

In [162]:
len(X_train), len(X_test), len(y_train), len(y_test)

(7000, 3000, 7000, 3000)

## Implementing the Random Forest Classifier Model

In [163]:
## trying the model for customers who will leave the bank

In [164]:
RF = RandomForestClassifier(n_estimators = 100, max_depth = 2, random_state = 0)

RF.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [170]:
## Saving the trained model to a file

model_filename = 'random_forest_model.pkl'
joblib.dump(RF, model_filename)

['random_forest_model.pkl']

In [171]:
## Downloading the model
import shutil
shutil.move('random_forest_model.pkl', 'random_forest_model_download.pkl')

'random_forest_model_download.pkl'

## Performance Check for Random Forest classifier

In [172]:
round(RF.score(X_train, y_train), 4)

0.8093

In [173]:
## Training we 80.93% accuracy

In [174]:
round(RF.score(X_test, y_test), 4)

0.8237

In [175]:
## Testing we get a 82.37% accuracy

## Checking Feature Importance for Random Forest Classifier Model

In [176]:
## Here is where we use eli5 in getting feature importance

In [177]:
perm = PermutationImportance(RF, random_state = 42, n_iter = 10).fit(X, y)

eli5.show_weights(perm, feature_names = X.columns.tolist())

Weight,Feature
0.0196 ± 0.0014,NumOfProducts
0.0070 ± 0.0011,Age
0.0012 ± 0.0008,Balance
0.0003 ± 0.0003,Geography
0.0002 ± 0.0006,Gender
0.0002 ± 0.0000,CreditScore
0.0002 ± 0.0005,IsActiveMember
0 ± 0.0000,HasCrCard
0 ± 0.0000,Tenure
-0.0000 ± 0.0002,EstimatedSalary


In [178]:
## This now indicates that NumOfProducts(age and balance) are our Top features

## Implementing an MLP Classifier Model

In [199]:
## Creating another training and testing set for another model

In [200]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, y, test_size = 0.30, random_state = 42)                                                                    

In [201]:
## Checking out with MLP classifier after trying out RandomForest

In [202]:
clf = MLPClassifier(random_state = 1, max_iter = 100).fit(X_train_new, y_train_new)

In [220]:
## Saving the trained model to a file

model_filename = 'MlpClassifier_model.pkl'
joblib.dump(clf, model_filename)

['MlpClassifier_model.pkl']

In [221]:
## Downloading the model
import shutil
shutil.move('MlpClassifier_model.pkl', 'MlpClassifierModel_download.pkl')

'MlpClassifierModel_download.pkl'

## Performance check for MLP Classifier model

In [183]:
clf.score(X_train, y_train_new)

0.7382857142857143

In [184]:
clf.score(X_test_new, y_test_new)

0.733

In [185]:
## Getting a 73.82% training accuracy and 73.30% testing accuracy

## Checking Feature Importance for MLP Classifier Model

In [186]:
perm = PermutationImportance(clf, random_state = 42, n_iter = 10).fit(X, y)

eli5.show_weights(perm, feature_names = X.columns.tolist())

Weight,Feature
0.0113 ± 0.0074,Balance
0.0037 ± 0.0024,Age
0.0002 ± 0.0006,IsActiveMember
0.0001 ± 0.0011,Tenure
0.0000 ± 0.0003,Gender
0 ± 0.0000,HasCrCard
0 ± 0.0000,NumOfProducts
-0.0000 ± 0.0006,Geography
-0.0012 ± 0.0020,CreditScore
-0.0016 ± 0.0051,EstimatedSalary


In [187]:
## This model indicates that Balance, Age and IsActiveMember to be our top features

## Implementing a NeuralNetwork Model and Checking the Importance

In [206]:
## Trying the Neural Network Model after trying out the Random forest and MLP classfiers and for that we'll use keras

In [222]:
model = keras.Sequential([
    keras.layers.Dense(10, input_shape = (10,), activation = 'relu'),
    keras.layers.Dense(25, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(optimizer = 'adam',
loss = 'binary_crossentropy',
metrics = ['accuracy'])

In [223]:
## We have a 25 node hidden layer. You can tweak and try out other combinations. We are using the 'adam' optimizer and 'binary_crossentropy' loss

In [224]:
## Fitting the model with 50epochs

In [225]:
model.fit(X_train, y_train, epochs = 50)

Epoch 1/50
219/219 [==============================] - 1s 2ms/step - loss: 537.6730 - accuracy: 0.6450
Epoch 2/50
219/219 [==============================] - 1s 2ms/step - loss: 15.7967 - accuracy: 0.6894
Epoch 3/50
219/219 [==============================] - 0s 2ms/step - loss: 12.7449 - accuracy: 0.6933
Epoch 4/50
219/219 [==============================] - 0s 2ms/step - loss: 10.8424 - accuracy: 0.6919
Epoch 5/50
219/219 [==============================] - 1s 2ms/step - loss: 15.3052 - accuracy: 0.6877
Epoch 6/50
219/219 [==============================] - 1s 2ms/step - loss: 16.7672 - accuracy: 0.6801
Epoch 7/50
219/219 [==============================] - 1s 2ms/step - loss: 10.0155 - accuracy: 0.6886
Epoch 8/50
219/219 [==============================] - 1s 2ms/step - loss: 13.8247 - accuracy: 0.6949
Epoch 9/50
219/219 [==============================] - 1s 2ms/step - loss: 15.3372 - accuracy: 0.6943
Epoch 10/50
219/219 [==============================] - 1s 2ms/step - loss: 12.6785 - accur

In [226]:
## After training the 50 epochs, I got a 79.24% accuracy.

In [227]:
model.evaluate(X_test, y_test)

94/94 [==============================] - 0s 2ms/step - loss: 3.0284 - accuracy: 0.8053


[3.028397798538208, 0.8053333163261414]

In [228]:
## After testing the model gives an 80.53% accuracy

In [229]:
## Printing the classification report and checking the perfomance

In [230]:
yp = model.predict(X_test)
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89      2416
           1       0.50      0.00      0.01       584

    accuracy                           0.81      3000
   macro avg       0.65      0.50      0.45      3000
weighted avg       0.75      0.81      0.72      3000



In [231]:
## Based on these metrics, we see that precision & recall are less with the above model for class 1 but good for class 0

In [232]:
## Saving the trained keras model in HDF5 format
model.save('keras_neural_network_model.h5')

##Downloading the saved model
shutil.move('keras_neural_network_model.h5', 'keras_neural_network_model_download.h5')

'keras_neural_network_model_download.h5'